In [7]:
import cv2
import sys
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
if __name__ == '__main__':

  if len(sys.argv) < 2:
    print('Usage: python ocr_simple.py image.jpg')
    sys.exit(1)
  
  # Read image path from command line
  imPath = sys.argv[1]
    
  # Uncomment the line below to provide path to tesseract manually
  # pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

  # Define config parameters.
  # '-l eng'  for using the English language
  # '--oem 1' sets the OCR Engine Mode to LSTM only.
  config = ('-l eng --oem 1 --psm 3 --psm 13')

  # Read image from disk
  im = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.png", cv2.IMREAD_COLOR)

  # Run tesseract OCR on image
  text = pytesseract.image_to_string(im)

  # Print recognized text
  print(text)

Company Name INVOICE

MrHamsho Company

George 3 Street INVOICE # 51
City, ST ZIP Code DATE 30.09.2022
Phone Enter phone | Fax Enter fax

Email | Website

JOHN FINGS

KTS Limited

Washington str

NYC 88221

+123 555752 JOHN@KTS.com

 

Description Unit Price Amount Total Pirce
Resin Printer M821 389 ] 389
FDM Printer Z7432 1099 ] 1099
PLA Filement 1KG Black 40 2 80
PLA Filement 1KG WHITE 40 ] 40
PLA Filement 1KG RED 40 ] 40
PLA Filement 1KG 40 2 80
Resin White 1L 69 ] 69
Resin Black 0.5L 39 ] 39
Resin Skin 1L 73 ] 73
1909 11

Make all checks payable to Company Name
Payment is due within 30 days.
If you have any questions concerning this invoice, contact Name | Phone | Email

THANK YOU FOR YOUR BUSINESS!



In [9]:
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\invoice.png")

d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(d.keys())

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 40:
        x, y, w, h = d['left'][i], d['top'][i], d['width'][i], d['height'][i]
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 1)

cv2.imshow('img', img)
cv2.waitKey(0)

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


-1

In [ ]:
print((d['text'][8]))
print((d['conf'][8]))
print((d['word_num'][8]))
for i in range(0,len(d['text'])):
    if d['word_num'][i]>0:
        print(d['text'][i])

In [ ]:
from __future__ import print_function
import cv2
import numpy as np

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)

  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))

  return im1Reg, h

if __name__ == '__main__':

  # Read reference image
  refFilename = "D:\\My Files\\Courses\\Python\\67_OCR\\invoice.png"
  print("Reading reference image : ", refFilename)
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

  # Read image to be aligned
  imFilename = "D:\\My Files\\Courses\\Python\\67_OCR\\invoice3.png"
  print("Reading image to align : ", imFilename);
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

  print("Aligning images ...")
  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
  imReg, h = alignImages(im, imReference)

  # Write aligned image to disk.
  outFilename = "D:\\My Files\\Courses\\Python\\67_OCR\\alligned.png"
  print("Saving aligned image : ", outFilename);
  cv2.imwrite(outFilename, imReg)

  # Print estimated homography
  print("Estimated homography : \n",  h)

In [ ]:
from pdf2image import convert_from_path
img = convert_from_path('D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.pdf')
img[0].save('D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.jpg', 'JPEG')
img=img[0]
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.jpg")
config = ('-l eng --oem 1 --psm 3 --psm 13')
 
d = pytesseract.image_to_data(img, output_type=Output.DICT,config=config)
print(d.keys())

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 40:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 1)

cv2.imshow('img', img)
cv2.waitKey(0)

In [86]:
import cv2
import cv2
import sys
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
config = ('-l eng --oem 1  --psm 3 --psm 6')
img = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.jpg")
#x,y,w,h
cord={'CompanyName':[80,193,340,70],
      'product':[90,790,660,48],
      'amount':[1250,790,200,48],
      'price':[1480,790,140,48]
     }
name='amount'
x=cord[name][0]
y=cord[name][1]
w=cord[name][2]
h=cord[name][3]

for i in range (0,6):
    
    crop_img = img[y:y+h, x:x+w]
    y=y+50
    
 
    cv2.imwrite("D:\\My Files\\Courses\\Python\\67_OCR\\crop"+str(i)+".jpg", crop_img)


import pytesseract
from pytesseract import Output

img = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\crop0.jpg")
img = cv2.GaussianBlur(img, (5, 5), 0)
d = pytesseract.image_to_data(img, output_type=Output.DICT, config=config)
print(d.keys())

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 40:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 1)
print(d['text'])
#cv2.imshow('img', img)
#cv2.waitKey(0)

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])
['', '', '', '', '1']


In [ ]:
print(d.keys())

In [25]:
import cv2
import pandas as pd
import sys
import pytesseract
from pdf2image import convert_from_path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
config = ('-l eng --oem 1 --psm 3 --psm 13')

img=None
df=None
cord={'CompanyName':[80,193,340,70],
      'product':[90,790,660,48],
      'amount':[1250,790,200,48],
      'price':[1480,790,140,48]
     }
dataDic = { "productNamePath":[],
            "productQuantityPath":[],
            "productIncomePath":[],
             "productName":[],
             "productQuantity":[],
             "productIncome":[]}
def convertPDF2Img():
    
    img = convert_from_path('D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.pdf')
    img[0].save('D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.jpg', 'JPEG')
    
def readInvoice():
    global img
    img = cv2.imread("D:\\My Files\\Courses\\Python\\67_OCR\\invoice1.jpg")
    
def extractProductInfo(info, numberOfFields, step, keyToStore):
    name=info
    x=cord[name][0]
    y=cord[name][1]
    w=cord[name][2]
    h=cord[name][3]
    for i in range (0,numberOfFields):  
        crop_img = img[y:y+h, x:x+w]
        y=y+step 
        path = "D:\\My Files\\Courses\\Python\\67_OCR\\"+str(info)+str(i)+".jpg"
        dataDic[keyToStore].append(path)
        cv2.imwrite(path, crop_img)
def OCR(path,key):
    for imgPath in dataDic[path]:
        img = cv2.imread(imgPath)
        
        if path=="productNamePath":
            config = ('-l eng --oem 1 --psm 3 ')
        else :
            config = ('-l eng --oem 1  --psm 6')
            img = cv2.GaussianBlur(img, (5, 5), 0)
        d = pytesseract.image_to_data(img, output_type=Output.DICT, config=config)
        #skip if page is empty or giving empty strings
        
        #check if list contains ONLY empty tring
        if not any(d['text']): 
            continue
        
            


       # print(dataDic[key])
        print(d['text'])
        #before this followign statemnet d['text']=['', '', '', '', 'Resin', 'Printer', 'M821']
        d['text'] = [x for x in d['text'] if x]
        #after that above statemnet d['text']=['Resin', 'Printer', 'M821']
        print(d['text'])
        
        #extract keyword from the list, and create a sentence seperated by a space
        s=' '.join(d['text'])
        
        dataDic[key].append(s)
        print(dataDic[key])
        

    
def dbIntegrityCheck():
    l=[]
    for v in  dataDic.values():
        l.append(len(v))
    print("Integrity is:", all(x == l[0] for x in l))
    print(l)
def getDB(path):
    global df
    df = pd.read_csv(path)

def getDBitemIndex(entry):
 
    return df.index[df['Item']==entry]

def subtractQuantity(quantity, index):
    global df
    df["Quantity"][index]=int(df["Quantity"][index])- int(quantity)

def addProductIncome(income, index):
    global df
    df["Income"][index]=int(df["Income"][index])+ int(income)

def getQuantity(index):
    return dataDic["productQuantity"][index]

def getIncome(index):
    return dataDic["productIncome"][index]

def proccesDB():
    for i,item in enumerate(dataDic['productName']):
        index=getDBitemIndex(item)
       
        print(index)
        subtractQuantity(getQuantity(i),index)
        addProductIncome(getIncome(i),index)
    
    
convertPDF2Img()
readInvoice()
extractProductInfo("product", 17, 50, "productNamePath")
extractProductInfo("amount", 17, 50, "productQuantityPath")
extractProductInfo("price", 17, 50, "productIncomePath")
OCR("productNamePath","productName")
OCR("productQuantityPath","productQuantity")
OCR("productIncomePath","productIncome")
 
getDB("D:\\My Files\\Courses\\Python\\67_OCR\\FinancialSheet.csv")
proccesDB()
df.to_csv("D:\\My Files\\Courses\\Python\\67_OCR\\out.csv", index=False)
#index=getDBitemIndex("Resin Skin 1L")
#subtractQuantity(17,index[0]+1)
#addProductIncome(230,index[0]+1)
#display(df)

['', '', '', '', 'Resin', 'Printer', 'M821']
['Resin', 'Printer', 'M821']
['Resin Printer M821']
['', '', '', '', 'FDM', 'Printer', 'Z7432']
['FDM', 'Printer', 'Z7432']
['Resin Printer M821', 'FDM Printer Z7432']
['', '', '', '', 'PLA', 'Filement', '1KG', 'Black']
['PLA', 'Filement', '1KG', 'Black']
['Resin Printer M821', 'FDM Printer Z7432', 'PLA Filement 1KG Black']
['', '', '', '', 'PLA', 'Filement', '1KG', 'WHITE']
['PLA', 'Filement', '1KG', 'WHITE']
['Resin Printer M821', 'FDM Printer Z7432', 'PLA Filement 1KG Black', 'PLA Filement 1KG WHITE']
['', '', '', '', 'PLA', 'Filement', '1KG', 'RED']
['PLA', 'Filement', '1KG', 'RED']
['Resin Printer M821', 'FDM Printer Z7432', 'PLA Filement 1KG Black', 'PLA Filement 1KG WHITE', 'PLA Filement 1KG RED']
['', '', '', '', 'PLA', 'Filement', '1KG']
['PLA', 'Filement', '1KG']
['Resin Printer M821', 'FDM Printer Z7432', 'PLA Filement 1KG Black', 'PLA Filement 1KG WHITE', 'PLA Filement 1KG RED', 'PLA Filement 1KG']
['', '', '', '', 'Resin', 'Whit

C:\Users\Hamsho\AppData\Local\Temp/ipykernel_6888/1967449515.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Quantity"][index]=int(df["Quantity"][index])- int(quantity)
C:\Users\Hamsho\AppData\Local\Temp/ipykernel_6888/1967449515.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Income"][index]=int(df["Income"][index])+ int(income)


In [149]:
display(df)

,Item,Quantity,Base Price,Sell Price,Income
0,Resin Printer M821,29,220,389,389
1,FDM Printer Z7432,29,830,1099,1099
2,PLA Filement 1KG Black,68,12,40,80
3,PLA Filement 1KG WHITE,69,12,40,40
4,PLA Filement 1KG RED,69,12,40,40
5,PLA Filement 1KG BLUE,70,12,40,0
6,PLA Filement 1KG GREY,70,12,40,0
7,PLA Filement 1KG BROWN,70,12,40,0
8,PLA Filement 1KG,68,12,40,80
9,Resin White 1L,24,39,69,69
